In [ ]:
import App.dataset as dataset
import App.visualisation as visualisation
from App.config import WIDTH, HEIGHT, NUMBER_OF_CHANNELS, Z_SCALE, RANDOM_MIN_VALUE, RANDOM_MAX_VALUE, BATCH_SIZE, BOTTLENECK_NUMBER_OF_FEATURE, INPUT_SHAPE

import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm

# Creation du Dataset

### 1. Créer un jeu de données en appliquant les prép-traitements nécessaires à l'AutoEncoder

In [ ]:
# Chemin vers le dossier contenant les fichiers LiDAR
folder_path = "Data/dataset/sequences/00/velodyne/"
# Créer le dataset
dataset_lidar_00 = dataset.create_bev_dataset_from_folder(folder_path, batch_size=BATCH_SIZE)

In [ ]:
# Chemin vers le dossier contenant les fichiers LiDAR
folder_path = "Data/dataset/sequences/00/velodyne/"
# Créer le dataset
dataset_lidar_01 = dataset.create_bev_dataset_from_folder(folder_path, batch_size=BATCH_SIZE)

In [ ]:
# Chemin vers le dossier contenant les fichiers LiDAR
folder_path = "Data/dataset/sequences/01/velodyne/"
# Créer le dataset
validation_dataset = dataset.create_bev_dataset_from_folder(folder_path, batch_size=BATCH_SIZE)

### 2. Visualisation de quelques exemples de données


In [ ]:
# Exemple d'utilisation avec une image

lidar_data = dataset.read_kitti_lidar("Data/dataset/sequences/00/velodyne/000000.bin")
bev = dataset.lidar_to_bev(lidar_data, bev_height=HEIGHT, bev_width=WIDTH, z_scale=Z_SCALE)
# Remplacer les zéros par des valeurs aléatoires
bev_with_random = dataset.replace_zeros_with_random(bev, min_value=RANDOM_MIN_VALUE, max_value=RANDOM_MAX_VALUE)

# Afficher les canaux BEV
visualisation.show_bev(bev_with_random, channel=0, title="BEV - Intensité")
visualisation.show_bev(bev_with_random, channel=1, title="BEV - Hauteur")
visualisation.show_bev(bev_with_random, channel=2, title="BEV - Densité")

# Afficher les nuages de points LiDAR
visualisation.lidar_point_clouds = [lidar_data]
visualisation.display_lidar_point_cloud("Data/dataset/sequences/00/velodyne/000000.bin")

# Création Auto Encoder

In [ ]:
import App.autoencoder as autoencoder

In [ ]:
# Création du modèle AutoEncoder BEV
bev_autoencoder = autoencoder.create_bev_autoencoder(
    input_shape=INPUT_SHAPE,
    bottleneck=BOTTLENECK_NUMBER_OF_FEATURE
)
# Affichage du résumé du modèle AutoEncoder
bev_autoencoder.summary()

# Apprentissage Machine

In [ ]:
# Train the AutoEncoder model
def train_bev_autoencoder( 
    model,
    dataset,
    epochs=10,
    batch_size=BATCH_SIZE,
    learning_rate=0.0001,
    save_path='bev_autoencoder.h5'
):
    """
    Entraîne le modèle AutoEncoder BEV.
    
    :param model: Modèle AutoEncoder à entraîner.
    :param dataset: Dataset TensorFlow pour l'entraînement.
    :param epochs: Nombre d'époques pour l'entraînement.
    :param batch_size: Taille du batch pour l'entraînement.
    :param learning_rate: Taux d'apprentissage pour l'optimiseur.
    :param save_path: Chemin pour enregistrer le modèle entraîné.
    """
    
    # COmpile the model with absolute squared error loss and Adam optimizer
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mean_squared_error',
        metrics=['mae'] 
    )
    
    history = model.fit(dataset, epochs=epochs, batch_size=batch_size)
    
    # Enregistrement du modèle
    model.save(save_path)
    
    return history

In [ ]:
# Entraînement du modèle AutoEncoder BEV
history = train_bev_autoencoder(
    model=bev_autoencoder,
    dataset=dataset_lidar_00,
    epochs=10,
    batch_size=BATCH_SIZE,
    learning_rate=0.001,
    save_path='bev_autoencoder.h5'
)
# Affichage de l'historique d'entraînement
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [ ]:
# Entraînement du modèle AutoEncoder BEV
history = train_bev_autoencoder(
    model=bev_autoencoder,
    dataset=dataset_lidar_01,  # Utilisation du dataset créé précédemment
    epochs=10,
    batch_size=BATCH_SIZE,
    learning_rate=0.001,
    save_path='bev_autoencoder.h5'
)
# Affichage de l'historique d'entraînement
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

# Validation de l'auto Encoder

In [ ]:
# Validation du modèle AutoEncoder BEV
def validate_bev_autoencoder(model, dataset, num_samples=5):
    """
    Valide le modèle AutoEncoder BEV en affichant des exemples d'entrées et de sorties.
    
    :param model: Modèle AutoEncoder à valider.
    :param dataset: Dataset TensorFlow pour la validation.
    :param num_samples: Nombre d'échantillons à afficher.
    """
    for i, (input_data, target_data) in enumerate(dataset.take(num_samples)):
        if i >= num_samples:
            break
        reconstructed = model.predict(input_data)
        
        # Affichage des entrées et des reconstructions
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.title("Input BEV")
        plt.imshow(input_data[0])
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.title("Reconstructed BEV")
        plt.imshow(reconstructed[0])
        plt.axis('off')
        
        plt.show()

In [ ]:
# Validation du modèle AutoEncoder BEV
validate_bev_autoencoder(
    model=bev_autoencoder,
    dataset=validation_dataset,
    num_samples=5
)

In [ ]:
# Calcul la courbe de validation
def plot_validation_curve(model, dataset, num_samples=5):
    """
    Affiche la courbe de validation du modèle AutoEncoder BEV.
    
    :param model: Modèle AutoEncoder à valider.
    :param dataset: Dataset TensorFlow pour la validation.
    :param num_samples: Nombre d'échantillons à afficher.
    """
    losses = []
    for input_data, target_data in dataset.take(num_samples):
        reconstructed = model.predict(input_data)
        loss = tf.keras.losses.MeanSquaredError()
        loss = loss(target_data, reconstructed)
        losses.append(tf.reduce_mean(loss).numpy())
    
    plt.plot(losses)
    plt.title('Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Sample Index')
    plt.show()

# Affichage de la courbe de validation
plot_validation_curve(
    model=bev_autoencoder,
    dataset=validation_dataset,
    num_samples=5
)